## Import packages

In [1]:
import pandas as pd
from CFModel import CFModel
from CFModel import DeepModel

/home/dislab/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Define constants


In [18]:
RATINGS_CSV_FILE = './input/rating_result.csv'
USERS_CSV_FILE = './input/user_result.csv'
RECIPE_CSV_FILE = './input/recipe_result.csv'
MODEL_WEIGHTS_FILE = 'recipe_weights.h5'
K_FACTORS = 120
TEST_USER = 200

## Load Recipe data

In [19]:
ratings = pd.read_csv(RATINGS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'recipeid', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max() + 1
max_recipeid = ratings['recipeid'].drop_duplicates().max() + 1
print(len(ratings), 'ratings loaded.')

(1048575, 'ratings loaded.')


In [20]:
users = pd.read_csv(USERS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'nickname'])
print(len(users), 'descriptions of', max_userid, 'users loaded.')

(75061, 'descriptions of', 12936, 'users loaded.')


In [21]:
recipes = pd.read_csv(RECIPE_CSV_FILE, sep=',', encoding='utf-8', usecols=['recipeid', 'title', 'categories'])
print(len(recipes), 'descriptions of', max_recipeid, 'recipes loaded.')

(5101, 'descriptions of', 5023, 'recipes loaded.')


## Make recommendations for a given user

In [22]:
trained_model = DeepModel(max_userid, max_recipeid, K_FACTORS)

In [23]:
trained_model.load_weights(MODEL_WEIGHTS_FILE)

In [24]:
users[users['userid'] == TEST_USER]

,userid,nickname
200,200,dislab200


In [25]:
def predict_rating(userid, recipeid):
    return trained_model.rate(userid, recipeid)

In [26]:
user_ratings = ratings[ratings['userid'] == TEST_USER][['userid', 'recipeid', 'rating']]

In [27]:
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(recipes, 
                                                on='recipeid', 
                                                how='inner',
                                                suffixes=['_u', '_m']
                                               ).head(10)

,userid,recipeid,rating,prediction,title,categories
0,200,1632,5,5.000000,불닭볶음면이 우유에 빠진날 불닭까르보나라,"[간식/야식, 실생활 요리, 양식 요리, 가공식품 요리, 10분 요리, 자취 초보 ..."
1,200,779,5,5.000000,카모메식당의 그 오니기리 주먹밥 오니기리,"[김밥/롤/주먹밥/초밥, 실생활 요리, 일식 요리, 가공식품 요리, 자취 초보 요리..."
2,200,696,5,4.999998,알리오 올리오 알리오 올리오,"[서양식면, 손님 접대 요리, 양식 요리, 혼밥 요리, 볶음면요리]"
3,200,704,5,4.999999,"여행 도시락으로 최고, 누텔라 프렌치토스트 롤 누텔라 프렌치토스트 롤","[김밥/롤/주먹밥/초밥, 빵/케이크, 간식/야식, 나들이 요리, 가공식품 요리, 움..."
4,200,713,5,4.999998,오늘도 맹모닝 굿모닝 : 토마토 &amp,베이컨 잉글리시 머핀 잉글리시 머핀
5,200,720,5,5.000000,"놀이공원에서 인기짱, 츄러스 초코토핑 츄러스","[빵/케이크, 과자/쿠키, 사탕/초콜렛, 간식/야식, 나들이 요리, 자취 초보 요리..."
6,200,770,5,4.999998,5가지 재료로 만드는 꾸덕한 크림 스파게티 크림 파스타,"[서양식면, 간식/야식, 파티/명절요리, 양식 요리, 자취 초보 요리, 간단 재료 ..."
7,200,774,5,4.999999,"숙취해소의 직방, 달콤한 망고빙수 망고빙수","[아이스크림, 간식/야식, 과일 요리, 10분 요리, 자취 초보 요리, 움짤 레시피]"
8,200,785,5,4.999999,"집에서 즐기는 뉴욕, 매그놀리아 바나나 푸딩 바나나 푸딩","[과자/쿠키, 간식/야식, 양식 요리, 달걀/유제품 요리, 자취 초보 요리, 아이 간식]"
9,200,1224,5,5.000000,"쫠~깃한 한 점, 실패없다!! 비프 크림 수제비","[쇠고기, 실생활 요리, 퓨전요리, 움짤 레시피, 아이 간식, 볶음면요리]"


In [28]:
recommendations = ratings[ratings['recipeid'].isin(user_ratings['recipeid']) == False][['recipeid']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(recipes,
                                                 on='recipeid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(10)

,recipeid,prediction,title,categories
0,4159,5.000116,할머니표 시골 비빔국수 시골 비빔국수,"[실생활 요리, 한식 요리, 채소류 요리, 달걀/유제품 요리, 가공식품 요리, 5분..."
1,2001,5.000056,"도시락 반찬은 물론, 아이들간식으로 딱! 계란초밥 계란초밥","[김밥/롤/주먹밥/초밥, 간식/야식, 나들이 요리, 실생활 요리, 일식 요리, 자취..."
2,2060,5.000050,매콤달콤 입맛돋는 만두 강정 만두 강정,"[만두요리, 튀김요리, 간식/야식, 술안주, 실생활 요리, 한식 요리, 자취 초보 ..."
3,2847,5.000045,국물불닭 불닭,"[닭고기, 국/탕요리, 술안주, 한식 요리]"
4,4730,5.000045,밥 꾹꾹 말아먹어요! 짬뽕밥 짬뽕밥,"[수산물, 국/탕요리, 찌개요리, 술안주, 해장요리, 실생활 요리, 한식 요리, 중..."
5,3471,5.000037,리조토가 이렇게 쉬운 요리였나요~! 찹쌀리조토 찹쌀리조토,"[쌀밥/잡곡밥, 볶음밥/오므라이스, 손님 접대 요리, 파티/명절요리, 실생활 요리,..."
6,1102,5.000035,다이어트 요리에서 화려한 메인으로! 해물단호박찜,"[찜요리, 파티/명절요리, 한식 요리, 채소류 요리, 자취 초보 요리, 간편 메인반..."
7,801,5.000009,엄마가 만들어주는 스팸김밥&amp,참치롤 스팸 &amp
8,701,5.000008,어깨춤이 절로나오는 간단 도시락 치즈베이컨말이,"[햄/소시지, 구이요리, 요거트, 간식/야식, 나들이 요리, 달걀/유제품 요리, 1..."
9,1314,5.000008,달달함에 내가 솔로인 것도 잊는다! 국물떡볶이 떡볶이,"[볶음요리, 간식/야식, 한식 요리, 가공식품 요리, 자취 초보 요리]"
